![RDD key pair](media/18.sparkml.jpg)

# 11 - Spark Streaming

-   Procesamiento escalable, *high-throughput* y tolerante a fallos de flujos de datos

![Streaming Flow](media/19.streaming-flow.png)

-   Entrada desde muchas fuentes: Kafka, Flume, Twitter, ZeroMQ, Kinesis o sockets TCP

### Arquitectura de Spark Streaming

Abstracción principal: DStream (`discretized stream`).

-   Representa un flujo continuo de datos

![dstreams](media/20.dstreams.png)

Arquitectura *micro-batch*

-   Los datos recibidos se agrupan en batches
-   Los batches se crean a intervalos regulares (batch interval)
-   Cada batch forma un RDD, que es procesado por Spark
-   Adicionalmente: transformaciones con estado mediante
    -   Operaciones con ventanas
    -   Tracking del estado por cada clave


- Página de Spark Streaming: <https://spark.apache.org/streaming/>
- Documentación principal (de la última versión): <https://spark.apache.org/docs/latest/streaming-programming-guide.html>

In [1]:
!pip install pyspark

You are using pip version 9.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [7]:
# Create apache spark context
from pyspark import SparkContext
sc = SparkContext(master="local", appName="Mi app")

In [6]:
# Stop apache spark context
sc.stop()

## Spark Streaming: ejemplo de WordCount en red y sin estado

Para ejecutar el ejemplo:

   1. Ve al terminal desde el que iniciaste la máquina virtual con `vagrant up` y ejecuta la orden `vagrant ssh`
   2. Una vez en el terminal de la máquina virtual, usa netcat como un servidor en el puerto 9999

    `$ nc -lk 9999`

   2. Ejecuta el código PySpark que viene a continuación 

   3. Escribe líneas en el terminal del netcat, que serán recogidas y procesadas por el script
    - Escribe palabras repetidas, para comprobar que las cuenta bien

In [8]:
from pyspark.streaming import StreamingContext
from operator import add

sc.setLogLevel("WARN")

# Contexto Streaming con un batch interval de 5 s
ssc = StreamingContext(sc, 5)

# DStream que conecta a localhost:9999
lines = ssc.socketTextStream("localhost", 9999)

# Ejecuta un WordCount
counts = lines.flatMap(lambda line: line.split(" "))\
              .map(lambda word: (word, 1))\
              .reduceByKey(add)
              
counts.pprint()

ssc.start() # Inicia la computacion
ssc.awaitTerminationOrTimeout(60) # Espera a que termine (acaba en 60 segundos)

Py4JJavaError: An error occurred while calling o791.start.
: java.lang.IllegalStateException: Only one StreamingContext may be started in this JVM. Currently running StreamingContext was started atorg.apache.spark.streaming.api.java.JavaStreamingContext.start(JavaStreamingContext.scala:556)
sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
java.lang.reflect.Method.invoke(Method.java:498)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
py4j.Gateway.invoke(Gateway.java:282)
py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
py4j.commands.CallCommand.execute(CallCommand.java:79)
py4j.GatewayConnection.run(GatewayConnection.java:238)
java.lang.Thread.run(Thread.java:748)
	at org.apache.spark.streaming.StreamingContext$.org

## Spark Streaming: ejemplo de WordCount en red con estado

Repite los pasos anteriores, ejecutando el siguiente código

 - Comprueba que el número de palabras se acumula entre accesos

In [4]:
from pyspark.streaming import StreamingContext
from operator import add

sc.setLogLevel("WARN")

# Contexto Streaming con un batch interval de 5 s
ssc = StreamingContext(sc, 5)

# DStream que conecta a localhost:9999
lines = ssc.socketTextStream("localhost", 9999)

ssc.checkpoint("/tmp/cpdir") # Activa checkpoint

def updateFunc(new_values, last_sum):
    return sum(new_values) + (last_sum or 0)

counts = lines.flatMap(lambda line: line.split(" "))\
              .map(lambda word: (word, 1))\
              .updateStateByKey(updateFunc)

counts.pprint()

ssc.start() # Inicia la computacion
ssc.awaitTerminationOrTimeout(60) # Espera a que termine (acaba en 60 segundos)

Py4JJavaError: An error occurred while calling o525.start.
: java.lang.IllegalStateException: Only one StreamingContext may be started in this JVM. Currently running StreamingContext was started atorg.apache.spark.streaming.api.java.JavaStreamingContext.start(JavaStreamingContext.scala:556)
sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
java.lang.reflect.Method.invoke(Method.java:498)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
py4j.Gateway.invoke(Gateway.java:282)
py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
py4j.commands.CallCommand.execute(CallCommand.java:79)
py4j.GatewayConnection.run(GatewayConnection.java:238)
java.lang.Thread.run(Thread.java:748)
	at org.apache.spark.streaming.StreamingContext$.org